In [ ]:
### Nutze klassifizierte Bilder um sie in CVAT zu labeln -> Polygon Label ###
### Exportiere das Dataset nicht als yolo sondern "CVAT for images 1.1" ###

### Dataset wird entpackt, xml eingelesen und in yolo Format txt gespeichert, verschieben der jpg & txt dateien in Zielordner, splitte sie ###

import os
import shutil
import xml.etree.ElementTree as ET
from zipfile import ZipFile
import splitfolders

# Set folder paths
base_folder = 'datasets/labels/segmentation/cvat_01'
img_folder = os.path.join(base_folder, 'images')
anno_folder = os.path.join(base_folder, 'sort/obj_train_data')
split_folder = os.path.join(base_folder, 'split')

# Extract all files from the archive
with ZipFile('datasets/images/segmentation/seg_task_cvat.zip', 'r') as zf:
    zf.extractall(base_folder)

# Set the path to the XML file
xml_file = os.path.join(base_folder, 'annotations.xml')

# Set the target directory for the output files
os.makedirs(anno_folder, exist_ok=True)

# Loop through all images in the XML file
for image in ET.parse(xml_file).getroot().findall('image'):
    # Get the image file name and dimensions
    file_name, width, height = image.get('name'), int(image.get('width')), int(image.get('height'))
    
    # Loop through all polygons in the image
    for polygon in image.findall('polygon'):
        # Get the label
        label = polygon.get('label')
        
        # Check if the label is "Dolphin_segment"
        if label == 'Dolphin_segment':
            # Normalize the points
            points_normalized = [f"{float(x)/width:.6f} {float(y)/height:.6f}" for x, y in map(str.split, polygon.get('points').split(';'))]
            
            # Create the output TXT file name
            txt_file_name = os.path.splitext(file_name)[0] + '.txt'
            
            # Write the class and normalized points to the output TXT file
            with open(os.path.join(anno_folder, txt_file_name), 'w') as f:
                f.write(f"0 {' '.join(points_normalized)}")
            
            # Copy the image file to the target directory
            shutil.copy(os.path.join(img_folder, file_name), anno_folder)

# Split the images into train/val/test folders
splitfolders.ratio(img_folder, output=split_folder, seed=42, ratio=(.8, .2, .0), group_prefix=2)

# Remove the original images folder
try:
    os.remove(img_folder)
except OSError:
    pass

In [ ]:
### trainiere das Modell ###

from ultralytics import YOLO
import cv2

# Laden des YOLOv8-Modells
model = YOLO("customize/model/yolov8x-seg.pt") #lade Model

# Trainieren des Modells
model.train(data="customize/data/seg_cvat.yaml", epochs=20, batch=2, device=0, seed=42, project='Output/Pfad', name='Projektname', pretrained=True, imgsz=640, save=True, workers=2, task='segment')

In [ ]:
### Vorhersage auf unbekannte Ausgeschnittene Delfin Bilder ###

model = YOLO("Pfad/zu/trainierten/Model/best.pt")  # load a custom model
model.predict("Pfad/zu/Bilderordner", box=False, hide_labels=True, save_txt=True, save=True, conf=0.5)